# Resume OCR

### Enviroment Setup

In [ ]:
import numpy as np
import cnocr
import jieba
#from PIL import Image

import cv2

from perspective import correct_perspective

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (60, 60)

### explore the data

In [ ]:

test_image_path = r"C:\Localdata\GitHub\ResumeOCR\Resume_Info_Extractor_Competition_Images\28.jpg"

image = cv2.imread(test_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

print(type(image), image.shape)
plt.imshow(image)
plt.show()


gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
print(type(gray_image), gray_image.shape)
plt.imshow(gray_image, cmap="gray")
plt.show()

blurred = cv2.GaussianBlur(gray_image, (3, 3), 0)
plt.imshow(blurred, cmap="gray")
plt.show()


In [ ]:
test = correct_perspective(image, intermediate=False)
plt.imshow(test)

In [ ]:
inverted = cv2.bitwise_not(blurred)
plt.imshow(inverted, cmap="gray")
plt.show()
cannyed  = cv2.Canny(inverted, 100, 255)
plt.imshow(cannyed, cmap="gray")
plt.show()


In [ ]:
lines = cv2.HoughLinesP(cannyed, 1, np.pi/180, 250)

for i in range(int(np.size(lines)/4)):
    for x1, y1, x2, y2 in lines[i]:
        cv2.line(image.copy(), (x1, y1), (x2, y2), (255, 255, 0), 3)
        
plt.imshow(image)
plt.show()

In [ ]:
def to_cartesian(img, lines):
    """Convert lines in polar system to lines in Cartesian system."""
    height, width, _ = img.shape
    coff = max(height, width)
    cartesian = []
    for rho, theta in lines:
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a * rho, b * rho
        x1, y1 = int(x0 + coff * (-b)), int(y0 + coff * (a))
        x2, y2 = int(x0 - coff * (-b)), int(y0 - coff * (a))
        cartesian.append((x1, y1, x2, y2))
  
    return cartesian

def annotate_lines(img, lines):
    """Annotate lines in img."""
    annnotated = np.array(img)

    for x1, y1, x2, y2 in lines:
        cv2.line(annnotated, (x1, y1), (x2, y2), (0, 0, 255), 20)

    return annnotated.astype(np.float32)

def get_corners(lines, intersections):
    """Get corners among intersections of lines.
     Limitation: this can only handle len(lines) == 4
      """
      return np.array(list(set(i for j in intersections
                             for i in j if len(i) > 0)),
                  dtype=np.float32)[:4]